In [98]:
import math
from collections import defaultdict
import pandas as pd

def preprocess_query(query: str) -> list:
    """
    Preprocess the input query.
    This is a placeholder for more advanced preprocessing.
    """
    # Convert to lowercase and tokenize
    return query.lower().split()

def query_expansion(preprocessed_query: list) -> list:
    """
    TO DO TOMORROW : EMBEDDINGS ETC
    Expand the query with synonyms or related terms.
    This is a placeholder for more advanced query expansion techniques.
    """
    return preprocessed_query

def compute_bm25(token, index, doc_lengths, avg_doc_length, total_docs, k1=1.5, b=0.75):
    """
    Compute BM25 score for each token over all documents in which it appears.

    Args:
        token (str): token
        index (dict): inverted index
        doc_lengths (dict): dictionary of document lengths
        avg_doc_length (float): average document length in the corpus.
        total_docs (int): number of documents in the corpus.
        k1 (float): term saturation.
        b (float): length normalization.

    Returns:
        dict: A dictionary with document numbers as keys and BM25 scores as values.
    """
    if token not in index:
        return {}  # Token not in the index

    df = index[token]['df']  # doc frequency
    idf = math.log((total_docs - df + 0.5) / (df + 0.5) + 1)  # idf calculation

    scores = {}
    for docno, positions in index[token]['doc_info'].items():
        freq = len(positions)  # frequency is len of positions list
        doc_length = doc_lengths.get(docno, 0) #get the length of doc
        term_freq_component = freq * (k1 + 1) / (freq + k1 * (1 - b + b * (doc_length / avg_doc_length))) #the second component of formula
        scores[docno] = idf * term_freq_component

    return scores

def retrieval_function(query, index, doc_lengths, k1=1.5, b=0.75):
    """
    Retrieve documents based on a query using the BM25 scoring function.

    Args:
        query (str): The user query as a string.
        index (dict): The inverted index structure.
        doc_lengths (dict): A dictionary of document lengths ({docno: length}).
        k1 (float): BM25 hyperparameter for term saturation.
        b (float): BM25 hyperparameter for length normalization.

    Returns:
        list: A list of tuples (docno, score) sorted by descending score.
    """
    # Preprocess the query
    tokens = preprocess_query(query)  # returns a list of tokens

    # Calculate average document length
    total_docs = len(doc_lengths)
    avg_doc_length = sum(doc_lengths.values()) / total_docs if total_docs > 0 else 0

    # aggregate scores for all tokens in the query
    aggregated_scores = defaultdict(float)
    for token in tokens:
        token_scores = compute_bm25(token, index, doc_lengths, avg_doc_length, total_docs, k1, b)
        for docno, score in token_scores.items():
            aggregated_scores[docno] += score

    # Sort documents by score in descending order and return the top results
    #return only the first 100 - but we do have to calculate for all of them probably
    return sorted(aggregated_scores.items(), key=lambda x: x[1], reverse=True)[:100]


In [ ]:
query = "example query"

    # Perform retrieval
results = retrieval_function(query, inverted_index_positions, doc_lengths)

    # Print results
print("Top Results:", results)

In [53]:
#first option is to embed all the words anyway using this model and then extract the embeddings for the query
#this takes very long
"""
from transformers import AutoTokenizer, AutoModel
import torch
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base") 
"""
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [85]:
# Load a large language model (e.g., GPT-2 or GPT-3-like model)

# we can also explore co-occurence models etc? -> but then we need to store the bm25 table in memory
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

import numpy as np
from tqdm import tqdm  # For progress visualization

# Function to compute embeddings for a batch of words
def get_embeddings_batch(words):
    inputs = tokenizer(words, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    # Take the mean of the hidden states for each word in the batch
    return outputs.last_hidden_state.mean(dim=1).detach().numpy()

# Precompute embeddings for the vocabulary in batches
def precompute_vocab_embeddings(vocab, batch_size=32):
    embeddings = []
    word_list = list(vocab)
    total_words = len(word_list)

    for i in tqdm(range(0, total_words, batch_size), desc="Processing batches"):
        batch_words = word_list[i:i + batch_size]
        try:
            batch_embeddings = get_embeddings_batch(batch_words)
            embeddings.append(batch_embeddings)
        except Exception as e:
            print(f"Failed to compute embeddings for batch: {batch_words}, error: {e}")

    # Stack all embeddings into a single numpy array
    return np.vstack(embeddings), word_list

# Assuming `vocab` is a set or list of words
vocab = inverted_index_positions.keys()  # Get vocabulary from the inverted index
vocab_embeddings, word_list = precompute_vocab_embeddings(vocab, batch_size=100)


c:\Users\annie\anaconda3\anaconda\Lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_

In [1]:
#make embeddings for the vocabulary
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm  # For progress visualization

from data_loaders import *

class EmbeddingModel:
    def __init__(self, vocab:list, model_name="microsoft/codebert-base"):
        """
        Initialize the EmbeddingModel with the specified vocabulary and model.
        Args:
            vocab (list or set): A list or set of vocabulary words.
            model_name (str): The name of the pretrained model to load.
        """
        # Initialize and load tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModel.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.model.to(self.device)
        self.vocab = vocab  # Store vocab as a list for processing

        #precompute embeddings for the vocabulary in batches
        self.embeddings = self.precompute_vocab_embeddings(batch_size = 100)


    def get_embeddings_batch(self, words):
        """
        Computes embeddings for a batch of words.
        Args:
            words (list): A batch of words.
        Returns:
            np.ndarray: A 2D array of embeddings for the input words.
        """
        # Tokenize the input words
        inputs = self.tokenizer(words, return_tensors="pt", padding=True, truncation=True).to(self.device)
        with torch.no_grad():
            outputs = self.model(**inputs)
        # Return the mean-pooled embeddings
        return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

    def precompute_vocab_embeddings(self, batch_size=50):
        """
        Precomputes embeddings for all words in the vocabulary using batching.
        Args:
            batch_size (int): The size of each batch for processing.
        Returns:
            np.ndarray: A 2D array of word embeddings.
            list: A list of words corresponding to the embeddings.
        """
        embeddings = []
        total_words = len(self.vocab)

        for i in tqdm(range(0, total_words, batch_size), desc="Processing batches"):
            batch_words = self.vocab[i:i + batch_size]
            try:
                batch_embeddings = self.get_embeddings_batch(batch_words)
                embeddings.append(batch_embeddings)
            except Exception as e:
                print(f"Failed to compute embeddings for batch: {batch_words}, error: {e}")

        return np.vstack(embeddings)
    
    def get_embedding(self, word):
        """
        Computes the embedding for a single word.
        Args:
            word (str): The input word.
        Returns:
            np.ndarray: The embedding of the input word.
        """
        embedding = self.get_embeddings_batch([word])
        return embedding

    def find_similar_words(self, word, top_k=5):
        """
        Finds the top_k most similar words to the given word using cosine similarity
        Args:
            word (str): The target word.
            vocab_embeddings (np.ndarray): The precomputed embeddings of the vocabulary.
            word_list (list): The vocabulary list corresponding to the embeddings.
            top_k (int): The number of similar words to return.
        Returns:
            list: A list of tuples containing similar words and their cosine similarities.
        """
        target_embedding = self.get_embedding(word)
        # Compute cosine similarity in a vectorized manner
        similarities = cosine_similarity(target_embedding, self.embeddings)[0]
        # Get the top_k most similar words
        top_indices = np.argsort(similarities)[::-1][:top_k]

        #return the top similar words
        return [self.vocab[i] for i in top_indices]


# Example usage
if __name__ == "__main__":
    # Example vocabulary
    inverted_index_positions = Index() # this is preloaded data for now
    vocab = inverted_index_positions.vocab

    # Initialize the model
    embedding_model = EmbeddingModel(vocab)  #this will also precompute all the embeddings

    #experimenting to see similarities
    experiment_words = vocab[100:110]
    for experiment_word in experiment_words:
        print(experiment_word, embedding_model.find_similar_words(experiment_word))

Skipping invalid term line: ::3939
Skipping invalid term line: http://.:1
Skipping invalid term line: http://.appspot.com/static/css/foo.css,:1
Skipping invalid term line: http://3v4l.org/VC93o:1
Skipping invalid term line: http://72dpi.nl/klanten/makelaarsland/team/:1
Skipping invalid term line: http://activemq.apache.org/should-i-use-transactions.html:1
Skipping invalid term line: http://adit.io/posts/2013-04-17-functors,_applicatives,_and_monads_in_pictures.html:1
Skipping invalid term line: http://adodson.com/hello.js/#helloapi:1
Skipping invalid term line: http://adodson.com/hello.js/#oauth-proxy:1
Skipping invalid term line: http://advancedlinuxprogramming.com/:1
Skipping invalid term line: http://afgomez.es/blog/better-ajax-callbacks-with-jquery-promises/:1
Skipping invalid term line: http://agency.lastminute-hr.com/stranice/upisi_destinacije_unico.php:1
Skipping invalid term line: http://ajax.googleapis.com/ajax/libs/angularjs/1.2.26/angular-route.min.js:1
Skipping invalid term

c:\Users\annie\anaconda3\anaconda\Lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
c:\Users\annie\anaconda3\anaconda\Lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializati

"106A" ['"RENUKA"', '"mraa"', '"monsters2"', '"voyage88"', '"0987654321"']
"1089" ['"011110000100101"', '"0987654321"', '"monsters2"', '"monsters1"', '"RENUKA"']
"109" ['"WEBSITE"', '"WSANSWER"', '"old_school"', '"dokuman"', '"WhiteOne"']
"10D" ['"dokuman"', '"multiOption2"', '"equalSpacing"', '"PivotStyle1"', '"monsters2"']
"10MB" ['"monsters1"', '"explain2"', '"sqlite3"', '"monsters2"', '"myCarousel2"']
"10dp" ['"monsters2"', '"vivek"', '"dokuman"', '"monsters1"', '"droppable"']
"10xyz" ['1921489Z"', '"0b0101__101"', '"myCarousel2"', '"011110000100101"', '"prime239v3"']
"11" ['"blah1"', '"blah3"', '"monsters1"', '"sqlite3"', '"mraa"']
"112 ['"sound4"', '"rd1Orange"', '"sound1"', '"rd1Pink"', '"rd1Female"']
"11447_en ['"parallelogramAreaBase"', 'pi_m4th_root', '"dt_rateio_inicial"', '"dt_rateio_final"', '"odincategory38"']


In [97]:
word = list(vocab)[100]  # Convert dict_keys to a list

print(word)
similar_words = find_similar_words(word, vocab_embeddings, word_list)
print(f"Top words similar to '{word}':")
for w, score in similar_words:
    print(f"{w}: {score:.4f}")

account
Top words similar to 'account':
account: 0.9851
activetab: 0.9842
table_nam: 0.9836
ifram: 0.9833
updatelayout: 0.9832


In [56]:
index = {
    "example": {
        "df": 2,
        "doc_info": {
            "doc1": [5, 15, 30],  # Token "example" appears 3 times in doc1
            "doc2": [7, 25]       # Token "example" appears 2 times in doc2
        }
    },
    "query": {
        "df": 1,
        "doc_info": {
            "doc2": [10]          # Token "query" appears 1 time in doc2
        }
    }
}

    # Document lengths (total tokens per document)
doc_lengths = {
        "doc1": 100,  # Document 1 has 100 tokens
        "doc2": 150   # Document 2 has 150 tokens
    }
vocab = ["example", "query"]

    # Define the query string
query = "example query"

    # Perform retrieval
results = retrieval_function(query, index, doc_lengths)

    # Print results
print("Top Results:", results)

Top Results: [('doc2', 0.8806417664214886), ('doc1', 0.3198623803402712)]


In [72]:
#load the index given
inverted_index_positions = {}

with open("index.txt", 'r', encoding='utf-8', errors='replace') as file:
    current_term = None
    for line in file:
        line = line.rstrip()  # Remove trailing whitespace
        
        if not line:  # Skip blank lines
            continue
        
        if not line.startswith("\t"):  # A new term with its document frequency
            # Check if the line contains a valid term:df pair
            if ":" in line:
                term, df = line.split(":", 1)  # Split only on the first ":"
                current_term = term.strip()
                try:
                    inverted_index_positions[current_term] = {'df': int(df.strip()), 'doc_info': []}
                except ValueError:
                    print(f"Skipping invalid term line: {line}")
                    current_term = None
        elif current_term:  # Document information under the current term
            # Check if the line starts with a valid docno
            if ":" in line.strip("\t"):
                doc_info = line.strip("\t")  # Remove the tab
                try:
                    docno, positions_str = doc_info.split(":", 1)
                    positions = list(map(int, positions_str.split(",")))  # Convert positions to integers
                    inverted_index_positions[current_term]['doc_info'].append((docno.strip(), positions))
                except ValueError:
                    print(f"Skipping invalid doc_info line: {line}")



Skipping invalid term line: http://blog.whatsapp.com/index.php/2011/09/one-million/":64
Skipping invalid term line: http://blogs.msdn.com/b/webdev/archive/2012/09/26/workaround-for-html-closing-tag-problem.aspx":13
Skipping invalid term line: http://blogs.msdn.com/b/webdev/archive/2012/09/26/workaround-for-html-closing-tag-problem.aspx</a></p>:13
Skipping invalid term line: http://chaicode-3lads.rhcloud.com/":33
Skipping invalid term line: http://docs.oracle.com/javafx/2/api/javafx/scene/web/webengine.html":15
Skipping invalid term line: http://docs.oracle.com/javafx/2/api/javafx/scene/web/webengine.html</a></p>:15
Skipping invalid term line: http://docs.oracle.com/javase/1.4.2/docs/api/java/lang/integer.html#parseint(java.lang.string":46
Skipping invalid term line: http://docs.oracle.com/javase/1.4.2/docs/api/java/lang/integer.html#parseint(java.lang.string</a>)</p>:46
Skipping invalid term line: http://github.com/zeusdeux/chaicode":33
Skipping invalid term line: http://headjs.com/":2

In [79]:
vocab = inverted_index_positions.keys()

In [ ]:
#build L2R